<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/haystack2x-demos/haystack_rag_services_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This notebook showcases a novel application of Qdrant vector DB in the upcoming Haystack OpenAPI service-based Retriever-Augmented Generation (RAG) system. Given a user query, we retrieve the corresponding OpenAPI specification from Qdrant vector DB. The retrieved service specification is then seamlessly injected into the Haystack pipeline, enabling the dynamic invocation of selected API services. This innovative approach, integrating any OpenAPI spec service with LLMs, opens a new era for RAG systems to generate contextually rich responses from any openapi service.

## 1. Setup
This notebook demos Haystack 2.x service based RAG, using two openapi services: GitHub's compare_branches and SerperDev search API.

Let's install necessary libraries and import key modules to build the foundation for the subsequent steps.

In [ ]:
!pip install -q "grpcio-tools==1.42" sentence-transformers openapi3 jsonref qdrant-haystack git+https://github.com/deepset-ai/haystack.git

In [ ]:
import getpass
import os
import json
import requests
from typing import List, Dict, Any
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
from haystack_integrations.components.retrievers.qdrant import QdrantEmbeddingRetriever

from haystack import Pipeline, Document
from haystack.components.generators.utils import print_streaming_chunk
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.converters import OpenAPIServiceToFunctions, OutputAdapter
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.connectors import OpenAPIServiceConnector
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.others import Multiplexer
from haystack.dataclasses import ChatMessage, ByteStream

## 2. Setting up Qdrant vector DB, create indexing pipeline

In [ ]:
document_store = QdrantDocumentStore(
    path="./qdrant_23",
    index="Document",
    embedding_dim=768,
    recreate_index=True
  )

In [ ]:
def create_docs(functions, specs, system_messages, service_credentials):
  docs = []
  for function, spec, system_message, service_credential in zip(functions, specs, system_messages, service_credentials):
    d = Document(content=json.dumps(function),
                 meta={"spec": json.dumps(spec),
                       "system_message": system_message,
                       "service_credential": service_credential})
    print(d.meta)
    docs.append(d)
  return docs

In [ ]:
indexing_pipeline = Pipeline()
indexing_pipeline.add_component("fetcher", LinkContentFetcher())
indexing_pipeline.add_component("mx", Multiplexer(List[ByteStream]))
indexing_pipeline.add_component("spec_to_functions", OpenAPIServiceToFunctions())
indexing_pipeline.add_component("embedder", SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-mpnet-base-v2",
                                                                                 meta_fields_to_embed=["spec", "system_message", "service_credential"]))
indexing_pipeline.add_component("writer", DocumentWriter(document_store=document_store))
indexing_pipeline.add_component("a1", OutputAdapter("{{functions | create_docs(specs, system_messages, service_credentials)}}",
                                                     List[Document],
                                                      {"create_docs": create_docs}))

indexing_pipeline.add_component("a2", OutputAdapter("{{sources | json_objects}}",
                                                     List[Dict[str, Any]],
                                                      {"json_objects": lambda sources: [json.loads(s.to_string()) for s in sources]}))

indexing_pipeline.connect(sender="fetcher.streams", receiver="mx")
indexing_pipeline.connect(sender="mx", receiver="spec_to_functions.sources")
indexing_pipeline.connect(sender="mx", receiver="a2.sources")
indexing_pipeline.connect(sender="a2", receiver="a1.specs")
indexing_pipeline.connect(sender="spec_to_functions.functions", receiver="a1")
indexing_pipeline.connect(sender="a1.output", receiver="embedder.documents")
indexing_pipeline.connect(sender="embedder", receiver="writer")

In [ ]:
serper_dev_key = getpass.getpass("Enter serperdev api key:")
github_token = getpass.getpass("Enter your github token:")

services_auth = {"SerperDev":serper_dev_key, "Github API": github_token}

## 3. Index two openapi specs (Github and SerperDev), along with system prompts

In [ ]:
result = indexing_pipeline.run(data={"fetcher": {"urls":["https://bit.ly/github_compare","https://bit.ly/serper_dev_spec"]},
                                     "a1":{"service_credentials": [github_token, serper_dev_key],
                                           "system_messages": [requests.get("https://bit.ly/pr_auto_system").text, requests.get("https://bit.ly/serper_dev_system_prompt").text]
                                           }
                                     })

In [ ]:
result["writer"]

In [ ]:
retrieval_pipeline = Pipeline()
retrieval_pipeline.add_component("embedder", SentenceTransformersTextEmbedder(model="sentence-transformers/all-mpnet-base-v2"))
retrieval_pipeline.add_component("retriever", QdrantEmbeddingRetriever(document_store=document_store))
retrieval_pipeline.connect(sender="embedder", receiver="retriever")

## 4. API keys, set up simple authentication mechanism

In [ ]:
llm_api_key = getpass.getpass("Enter LLM provider api key:")


## 5. Retrieval step - service invocation

Based on the prompt, retrieve the right service spec from Qdrant, inject spec and system prompt into the pipeline

In [ ]:
from haystack.utils import Secret

invoke_service_pipe = Pipeline()
invoke_service_pipe.add_component("functions_llm", OpenAIChatGenerator(api_key=Secret.from_token(llm_api_key), model="gpt-3.5-turbo-0613"))
invoke_service_pipe.add_component("openapi_container", OpenAPIServiceConnector())
invoke_service_pipe.connect("functions_llm.replies", "openapi_container.messages")

In [ ]:
user_prompt = "Search web and tell me why was Sam Altman ousted from OpenAI"
user_prompt = "Compare branches main and test/benchmarks2.0, in project deepset-ai, repo haystack"

In [ ]:
top_k = 1
top_k_result = retrieval_pipeline.run(data={"text": user_prompt, "top_k": top_k})
top_k_docs = top_k_result["retriever"]["documents"][:top_k]

In [ ]:
top_1 = top_k_docs[0]
openai_functions_definition = json.loads(top_1.content)

tools_param = [{"type": "function", "function": openai_functions_definition}]
tool_choice = {"type": "function", "function": {"name": openai_functions_definition["name"]}}

service_response = invoke_service_pipe.run(data={"messages":[ChatMessage.from_user(user_prompt)],
                                                 "generation_kwargs": {"tools": tools_param,
                                                                       "tool_choice": tool_choice},
                                                 "service_openapi_spec": json.loads(top_1.meta["spec"]),
                                                 "service_credentials": top_1.meta["service_credential"]})

## 6. Generate LLM response

Inject service response into LLM context, pair it with system prompt

In [ ]:
gen_pipe = Pipeline()
llm = OpenAIChatGenerator(api_key=Secret.from_token(llm_api_key), model="gpt-4-1106-preview", streaming_callback=print_streaming_chunk)
gen_pipe.add_component("llm", llm)

github_pr_prompt_messages = [ChatMessage.from_system(top_1.meta["system_message"])] + service_response["openapi_container"]["service_response"]
final_result = gen_pipe.run(data={"messages": github_pr_prompt_messages})

In [ ]:
from IPython.display import display, Markdown
display(Markdown(final_result["llm"]["replies"][0].content))


## Thank you, questions?

<a href="www.qr-code-generator.com/" border="0" style="cursor:default" rel="nofollow"><img src="https://chart.googleapis.com/chart?cht=qr&chl=https%3A%2F%2Fgithub.com%2Fvblagoje%2Fnotebooks%2Fblob%2Fmain%2Fhaystack2x-demos%2Fhaystack_rag_services_demo.ipynb&chs=180x180&choe=UTF-8&chld=L|2"></a>

## Links:
- https://github.com/deepset-ai/haystack/
- https://haystack.deepset.ai/community
- https://haystack.deepset.ai/advent-of-haystack
- https://x.com/vladblagoje